In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import warnings
warnings.filterwarnings(action='ignore')
from catboost import *
import sklearn
import math
from sklearn.preprocessing import LabelEncoder
import math


def log(train,test):
    col_list = train.columns
    for col in col_list :
        if '제곱' in col:
            train[col+'log'] = np.log1p(train[col])
            test[col+'log'] = np.log1p(test[col])
    return train,test

def del_columns(train, test):
    col_list = train.columns
    nan_list = []
    nan_cnt = []
    nan_col = []
    full_list = []
    for col in col_list:
        if train[col].isnull().sum() == 0 :
            full_list.append(col)
            continue
        nan_list.append([col, train[col].isnull().sum()])
        nan_cnt.append(train[col].isnull().sum())
        nan_col.append(col)

    '''모든값이 결측값이면 제거'''
    del_col = []
    for col in nan_list :
        if col[1] == len(train) :
            del_col.append(col[0])
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)

    '''값이 1개 존재하면 제거'''
    del_col = []
    col_list = train.describe().columns
    for col in col_list :
        if col == 'Y_Class':
            continue
        if col == 'Y_Quality':
            continue
        if col == 'LINE':
            continue
        if col == 'PRODUCT_CODE':
            continue
        if train[col].nunique()==1 :
            del_col.append(col)
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)
    
    return train,test
   
def make_train_test_dataset(train,test):
    train_x = train.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
    test_x = test.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
    train_y = train['Y_Quality']
    train_w = train[['Y_Class']]
    return train_x, test_x, train_y, train_w

def fillna(train,test,value):
    train = train.fillna(value)
    test = test.fillna(value)
    return train,test

def labelencoder(train,test,col_list):
    qual_col = col_list
    for i in qual_col:
        le = LabelEncoder()
        le = le.fit(train[i])
        train[i] = le.transform(train[i])

        for label in np.unique(test[i]): 
            if label not in le.classes_: 
                le.classes_ = np.append(le.classes_, label)
        test[i] = le.transform(test[i]) 
    return train,test

def multi_col(train,test) : 
    col_list = train.columns
    for col in col_list :
        if col == 'LINE':
            continue
        train[col+'제곱'] = (train[col]**2)//100
        test[col+'제곱'] = (test[col]**2)//100
    return train,test

In [49]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# y quality scaling
train['Y_Quality'] = train['Y_Quality'].map(lambda x : np.log(x))

train, test = del_columns(train,test)

trainA_31 = train[train['PRODUCT_CODE']=='A_31']
train_T_31 = train[train['PRODUCT_CODE']=='T_31']
train_O_31 = train[train['PRODUCT_CODE']=='O_31']

testA_31 = test[test['PRODUCT_CODE']=='A_31']
test_T_31 = test[test['PRODUCT_CODE']=='T_31']
test_O_31 = test[test['PRODUCT_CODE']=='O_31']

trainA_31, testA_31 = del_columns(trainA_31,testA_31)
train_T_31, test_T_31 = del_columns(train_T_31,test_T_31)
train_O_31, test_O_31 = del_columns(train_O_31,test_O_31)

trainA_31_x,testA_31_x, trainA_31_y, trainA_31_w = make_train_test_dataset(trainA_31,testA_31)
train_T_31_x,test_T_31_x, train_T_31_y, train_T_31_w = make_train_test_dataset(train_T_31,test_T_31)
train_O_31_x,test_O_31_x, train_O_31_y, train_O_31_w = make_train_test_dataset(train_O_31,test_O_31)

trainA_31_x,testA_31_x = labelencoder(trainA_31_x,testA_31_x,['LINE'])
train_T_31_x,test_T_31_x = labelencoder(train_T_31_x,test_T_31_x,['LINE'])
train_O_31_x,test_O_31_x = labelencoder(train_O_31_x,test_O_31_x,['LINE'])

trainA_31_x,testA_31_x = fillna(trainA_31_x,testA_31_x,-1)
train_T_31_x,test_T_31_x = fillna(train_T_31_x,test_T_31_x,-1)
train_O_31_x,test_O_31_x = fillna(train_O_31_x,test_O_31_x,-1)

drop_a = ['X_1167','X_1168','X_1169','X_1170',
          'X_1171','X_1274','X_1275','X_1276','X_1277','X_1278']

drop_t = ['X_529','X_530','X_531','X_532','X_533','X_600','X_601','X_602','X_603','X_604']
'''
trainA_31_x,testA_31_x = multi_col(trainA_31_x,testA_31_x)
train_T_31_x,test_T_31_x = multi_col(train_T_31_x,test_T_31_x)
train_O_31_x,test_O_31_x = multi_col(train_O_31_x,test_O_31_x)'''

print(trainA_31_x.shape,testA_31_x.shape,train_T_31_x.shape,test_T_31_x.shape)

(249, 1866) (67, 1866) (343, 550) (239, 550)


In [36]:
model1.get_all_params()

{'nan_mode': 'Min',
 'eval_metric': 'RMSE',
 'iterations': 1500,
 'sampling_frequency': 'PerTree',
 'leaf_estimation_method': 'Newton',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'bayesian_matrix_reg': 0.10000000149011612,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': True,
 'model_size_reg': 0.5,
 'subsample': 1,
 'use_best_model': False,
 'random_seed': 1234,
 'depth': 6,
 'posterior_sampling': False,
 'border_count': 254,
 'classes_count': 0,
 'auto_class_weights': 'None',
 'sparse_features_conflict_fraction': 0,
 'leaf_estimation_backtracking': 'AnyImprovement',
 'best_model_min_trees': 1,
 'model_shrink_rate': 0,
 'min_data_in_leaf': 1,
 'loss_function': 'RMSE',
 'learning_rate': 0.032999999821186066,
 'score_function': 'Cosine',
 'task_type': 'CPU',
 'leaf_estimation_iterations': 1,
 'bootstrap_type': 'MVS',
 'max_leaves': 64}

In [67]:
model1 = CatBoostRegressor(random_state=1234,verbose=500,iterations=500,learning_rate=0.033,
                          depth = 10)
model2 = CatBoostRegressor(random_state=1234,verbose=500,iterations=4500,learning_rate=0.02)

In [68]:
model1.fit(trainA_31_x, trainA_31_y)
pred_a = model1.predict(testA_31_x)

0:	learn: 0.0183258	total: 901ms	remaining: 7m 29s
499:	learn: 0.0004987	total: 7m 30s	remaining: 0us


In [69]:
model1.fit(train_T_31_x, train_T_31_y)
pred_t = model1.predict(test_T_31_x)

0:	learn: 0.0089054	total: 403ms	remaining: 3m 20s
499:	learn: 0.0002770	total: 3m 4s	remaining: 0us


In [55]:
model1.fit(train_O_31_x, train_O_31_y)
pred_o = model1.predict(test_O_31_x)

0:	learn: 0.0059697	total: 4.61ms	remaining: 6.91s
500:	learn: 0.0000906	total: 2.77s	remaining: 5.52s
1000:	learn: 0.0000014	total: 5.53s	remaining: 2.75s
1499:	learn: 0.0000000	total: 9.06s	remaining: 0us


In [28]:
print(train[train['Y_Class']==0]['Y_Quality'].max(),'\n',
      train[train['Y_Class']==1]['Y_Quality'].min(),'\n',
      train[train['Y_Class']==1]['Y_Quality'].max(),'\n',
      train[train['Y_Class']==2]['Y_Quality'].min(),'\n')

-0.6442300396904909 
 -0.6441937649548493 
 -0.6257823004654807 
 -0.6255805101477584 



In [70]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

testA_31.loc[(testA_31['Y_quanlity']<-0.64421190232267), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']>-0.6257823004654807), 'Y_Class'] = 2
test_T_31.loc[(test_T_31['Y_quanlity']<-0.64421190232267), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']>-0.6257823004654807), 'Y_Class'] = 2
test_O_31.loc[(test_O_31['Y_quanlity']<-0.64421190232267), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']>-0.6257823004654807), 'Y_Class'] = 2

submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')
submito = pd.read_csv('sample_submission.csv')

submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('dddd.csv',index=False)

In [71]:
dddd = pd.read_csv('dddd.csv')
dddd['Y_Class'].value_counts()

1    296
0     13
2      1
Name: Y_Class, dtype: int64

In [58]:
dddd[dddd['Y_Class']==2]

,PRODUCT_ID,Y_Class
38,TEST_038,2


In [ ]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

testA_31.loc[(testA_31['Y_quanlity']<-1.3373772202504361 ), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']> -1.318929481025426 ), 'Y_Class'] = 2
test_T_31.loc[(test_T_31['Y_quanlity']<-1.3373772202504361 ), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']> -1.318929481025426 ), 'Y_Class'] = 2
test_O_31.loc[(test_O_31['Y_quanlity']<-1.3373772202504361 ), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']> -1.318929481025426 ), 'Y_Class'] = 2

submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')
submito = pd.read_csv('sample_submission.csv')

submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('wpqkf.csv',index=False)

In [4]:
print(train[train['Y_Class']==0]['Y_Quality'].max(),'\n',
      train[train['Y_Class']==1]['Y_Quality'].min(),'\n',
      train[train['Y_Class']==1]['Y_Quality'].max(),'\n',
      train[train['Y_Class']==2]['Y_Quality'].min(),'\n')

-1.3373772202504361 
 -1.3373409455147947 
 -1.318929481025426 
 -1.3187276907077037 



In [14]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

testA_31.loc[(testA_31['Y_quanlity']<-1.3373772202504361 ), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']> -1.318929481025426 ), 'Y_Class'] = 2
test_T_31.loc[(test_T_31['Y_quanlity']<-1.3373772202504361 ), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']> -1.318929481025426 ), 'Y_Class'] = 2
test_O_31.loc[(test_O_31['Y_quanlity']<-1.3373772202504361 ), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']> -1.318929481025426 ), 'Y_Class'] = 2

submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')
submito = pd.read_csv('sample_submission.csv')

submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('wpqkf.csv',index=False)

In [22]:
pd.read_csv('wpqkf.csv')['Y_Class'].value_counts()

1    269
0     35
2      6
Name: Y_Class, dtype: int64

In [23]:
pd.read_csv('wpqkf.csv')[pd.read_csv('wpqkf.csv')['Y_Class']==2]

,PRODUCT_ID,Y_Class
38,TEST_038,2
78,TEST_078,2
109,TEST_109,2
160,TEST_160,2
161,TEST_161,2
163,TEST_163,2


In [101]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

testA_31['Y_quanlity2'] = pred_a2
test_T_31['Y_quanlity2'] = pred_t2
test_O_31['Y_quanlity2'] = pred_o2

testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

testA_31.loc[(testA_31['Y_quanlity2']< -3.686216323396579 ), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']>-0.6255805101477584 ), 'Y_Class'] = 2
test_T_31.loc[(test_T_31['Y_quanlity2']< -3.686216323396579 ), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']>-0.6255805101477584 ), 'Y_Class'] = 2
test_O_31.loc[(test_O_31['Y_quanlity2']< -3.686216323396579 ), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']>-0.6255805101477584 ), 'Y_Class'] = 2

submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')
submito = pd.read_csv('sample_submission.csv')

submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('scale2.csv',index=False)

In [59]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# y quality scaling
train['Y_Quality'] = train['Y_Quality'].map(lambda x : np.log(x))

train, test = del_columns(train,test)

trainA_31 = train[train['PRODUCT_CODE']=='A_31']
train_T_31 = train[train['PRODUCT_CODE']=='T_31']
train_O_31 = train[train['PRODUCT_CODE']=='O_31']

testA_31 = test[test['PRODUCT_CODE']=='A_31']
test_T_31 = test[test['PRODUCT_CODE']=='T_31']
test_O_31 = test[test['PRODUCT_CODE']=='O_31']

trainA_31, testA_31 = del_columns(trainA_31,testA_31)
train_T_31, test_T_31 = del_columns(train_T_31,test_T_31)
train_O_31, test_O_31 = del_columns(train_O_31,test_O_31)

trainA_31_x,testA_31_x, trainA_31_y, trainA_31_w = make_train_test_dataset(trainA_31,testA_31)
train_T_31_x,test_T_31_x, train_T_31_y, train_T_31_w = make_train_test_dataset(train_T_31,test_T_31)
train_O_31_x,test_O_31_x, train_O_31_y, train_O_31_w = make_train_test_dataset(train_O_31,test_O_31)

trainA_31_x,testA_31_x = labelencoder(trainA_31_x,testA_31_x,['LINE'])
train_T_31_x,test_T_31_x = labelencoder(train_T_31_x,test_T_31_x,['LINE'])
train_O_31_x,test_O_31_x = labelencoder(train_O_31_x,test_O_31_x,['LINE'])

trainA_31_x,testA_31_x = fillna(trainA_31_x,testA_31_x,-1)
train_T_31_x,test_T_31_x = fillna(train_T_31_x,test_T_31_x,-1)
train_O_31_x,test_O_31_x = fillna(train_O_31_x,test_O_31_x,-1)

print(trainA_31_x.shape,testA_31_x.shape,train_T_31_x.shape,test_T_31_x.shape)

(249, 1866) (67, 1866) (343, 550) (239, 550)


In [58]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

testA_31.loc[(testA_31['Y_quanlity']< -3.686216323396579    ), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']<  -3.686216323396579  ), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']<  -3.686216323396579  ), 'Y_Class'] = 0

In [71]:
submita = pd.read_csv('s05.csv')
submitt = pd.read_csv('s05.csv')
submito = pd.read_csv('s05.csv')

In [72]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

In [ ]:
testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

testA_31.loc[(testA_31['Y_quanlity']< -3.686216323396579    ), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']<  -3.686216323396579  ), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']<  -3.686216323396579  ), 'Y_Class'] = 0

In [65]:
testA_31.loc[(testA_31['Y_quanlity']> -0.6257823004654807  ), 'Y_Class'] = 2
test_T_31.loc[(test_T_31['Y_quanlity']> -0.6257823004654807  ), 'Y_Class'] = 2
test_O_31.loc[(test_O_31['Y_quanlity']> -0.6257823004654807  ), 'Y_Class'] = 2

submita = pd.read_csv('s05.csv')
submitt = pd.read_csv('s05.csv')
submito = pd.read_csv('s05.csv')

submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('scale3.csv',index=False)

KeyError: 'Y_quanlity'

In [47]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

testA_31.loc[(testA_31['Y_quanlity']< -2.0787560646728904    ), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']> -2.003645200765986  ), 'Y_Class'] = 2
test_T_31.loc[(test_T_31['Y_quanlity']<  -2.0787560646728904  ), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']> -2.003645200765986  ), 'Y_Class'] = 2
test_O_31.loc[(test_O_31['Y_quanlity']<  -2.0787560646728904  ), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']> -2.003645200765986  ), 'Y_Class'] = 2

submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')
submito = pd.read_csv('sample_submission.csv')

submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('scale3.csv',index=False)

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# y quality scaling
train['Y_Quality'] = train['Y_Quality'].map(lambda x : np.log(x))

train, test = del_columns(train,test)

trainA_31 = train[train['PRODUCT_CODE']=='A_31']
train_T_31 = train[train['PRODUCT_CODE']=='T_31']
train_O_31 = train[train['PRODUCT_CODE']=='O_31']

testA_31 = test[test['PRODUCT_CODE']=='A_31']
test_T_31 = test[test['PRODUCT_CODE']=='T_31']
test_O_31 = test[test['PRODUCT_CODE']=='O_31']

trainA_31, testA_31 = del_columns(trainA_31,testA_31)
train_T_31, test_T_31 = del_columns(train_T_31,test_T_31)
train_O_31, test_O_31 = del_columns(train_O_31,test_O_31)

trainA_31_x,testA_31_x, trainA_31_y, trainA_31_w = make_train_test_dataset(trainA_31,testA_31)
train_T_31_x,test_T_31_x, train_T_31_y, train_T_31_w = make_train_test_dataset(train_T_31,test_T_31)
train_O_31_x,test_O_31_x, train_O_31_y, train_O_31_w = make_train_test_dataset(train_O_31,test_O_31)

trainA_31_x,testA_31_x = labelencoder(trainA_31_x,testA_31_x,['LINE'])
train_T_31_x,test_T_31_x = labelencoder(train_T_31_x,test_T_31_x,['LINE'])
train_O_31_x,test_O_31_x = labelencoder(train_O_31_x,test_O_31_x,['LINE'])

trainA_31_x,testA_31_x = fillna(trainA_31_x,testA_31_x,-1)
train_T_31_x,test_T_31_x = fillna(train_T_31_x,test_T_31_x,-1)
train_O_31_x,test_O_31_x = fillna(train_O_31_x,test_O_31_x,-1)

print(trainA_31_x.shape,testA_31_x.shape,train_T_31_x.shape,test_T_31_x.shape)

In [13]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

testA_31.loc[(testA_31['Y_quanlity']<-0.64421190232267), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']>-0.6257823004654807), 'Y_Class'] = 2
test_T_31.loc[(test_T_31['Y_quanlity']<-0.64421190232267), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']>-0.6257823004654807), 'Y_Class'] = 2
test_O_31.loc[(test_O_31['Y_quanlity']<-0.64421190232267), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']>-0.6257823004654807), 'Y_Class'] = 2

submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')
submito = pd.read_csv('sample_submission.csv')

submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('scale2.csv',index=False)

In [102]:
df3 = pd.read_csv('scale2.csv')

In [99]:
df = pd.read_csv('s05.csv')

In [105]:
df3['Y_Class'].value_counts()#/len(df3)

1    241
0     61
2      8
Name: Y_Class, dtype: int64

In [100]:
df['Y_Class'].value_counts()

1    246
0     61
2      3
Name: Y_Class, dtype: int64

In [31]:
df[df['Y_Class']==2]

,PRODUCT_ID,Y_Class
38,TEST_038,2
40,TEST_040,2
72,TEST_072,2
78,TEST_078,2
161,TEST_161,2
163,TEST_163,2
260,TEST_260,2


In [14]:
df2= pd.read_csv('scale2.csv')

In [15]:
df2[df2['Y_Class']==2]

,PRODUCT_ID,Y_Class
38,TEST_038,2
78,TEST_078,2
134,TEST_134,2
160,TEST_160,2
161,TEST_161,2
163,TEST_163,2
169,TEST_169,2
177,TEST_177,2
183,TEST_183,2
187,TEST_187,2


In [98]:
df3[df3['Y_Class']==2]

,PRODUCT_ID,Y_Class
38,TEST_038,2
78,TEST_078,2
134,TEST_134,2
160,TEST_160,2
161,TEST_161,2
163,TEST_163,2
169,TEST_169,2
177,TEST_177,2
183,TEST_183,2
187,TEST_187,2
